In [1]:
# 🔄 Auto-reload modules when their source code changes
%load_ext autoreload
%autoreload 2

# 📦 Set up src/ and utils/ paths (project root is detected via pyproject.toml)
%run ../bootstrap.py

In [2]:
import requests
import datetime
import json
from pathlib import Path
from dotenv import load_dotenv
import os
import time
from IPython.display import display, HTML
import ipywidgets as widgets
from IPython.display import display, clear_output

from sleeping_beauty.config.config import Config

In [3]:
load_dotenv()

True

In [4]:
config = Config()


PROJECT_ROOT = Path(config.PROJECT_ROOT)
config.config_path = str(PROJECT_ROOT / "configs" / "config.yaml")
# "configs/FE/features_config.yaml"
config.load_from_yaml(config.config_path)




[Config] Loaded YAML config: /Users/kenneth/Public/projects/python/observatory/sleeping-beauty/configs/config.yaml


In [5]:
config.print_config_info()

📂 Configuration
--------------------------------------------------
Configuration file:       /Users/kenneth/Public/projects/python/observatory/sleeping-beauty/configs/config.yaml
--------------------------------------------------
🔐 Auth / Oura
--------------------------------------------------
Token path:               /Users/kenneth/.sleeping_beauty/oura_token.json
Client ID set:            True
Client Secret set:        True
Redirect URI:             http://localhost:8400/callback
Scopes:                   ['daily', 'personal', 'session']


In [6]:
from datetime import date, timedelta
import asyncio

from sleeping_beauty.clients.oura_api_client import OuraApiClient
from sleeping_beauty.oura.auth.oura_auth import OuraAuth  
from sleeping_beauty.oura.auth.storage.file_storage import FileTokenStorage

In [7]:
storage = FileTokenStorage(
    path=Path(
        config.oura_token_path
    ).expanduser()
)

# Instantiate your existing auth service
oura_auth = OuraAuth.from_config()
oura_auth.preflight_check()

# Token provider callable (what the client expects)
token_provider = oura_auth.get_access_token   ## notice this is passing the get_access_token as the callable function 

# Create the API client
client = OuraApiClient(token_provider=token_provider)

🔐 Oura Auth Preflight Check
----------------------------------------
✅ client_id present
✅ client_secret present
✅ redirect_uri: http://localhost:8400/callback
✅ requested scopes: ['extapi:daily', 'extapi:personal', 'extapi:session']
✅ token found in storage
✅ token valid for ~43199 minutes
✅ token covers requested scopes
----------------------------------------
✔️  Preflight check complete


In [10]:
auth = OuraAuth.from_config()
auth.preflight_check()

sessions = []

async with OuraApiClient(token_provider=auth.get_access_token) as client:
    async for session in client.get_sessions(
        start_date="2026-01-01",
        end_date="2026-01-31",
    ):
        sessions.append(session)

len(sessions), sessions[:2]

🔐 Oura Auth Preflight Check
----------------------------------------
✅ client_id present
✅ client_secret present
✅ redirect_uri: http://localhost:8400/callback
✅ requested scopes: ['extapi:daily', 'extapi:personal', 'extapi:session']
✅ token found in storage
✅ token valid for ~43198 minutes
✅ token covers requested scopes
----------------------------------------
✔️  Preflight check complete


(0, [])

In [9]:
DOCUMENT_ID = sessions[0].id  # or paste manually


async def fetch_single_session(doc_id: str):
    auth = OuraAuth.from_config()
    auth.preflight_check()

    async with OuraApiClient(token_provider=auth.get_access_token) as client:
        return await client.get_session(doc_id)


session = asyncio.run(fetch_single_session(DOCUMENT_ID))
session

IndexError: list index out of range